In [ ]:
from datasets import Dataset
import pandas as pd

def read_conll(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        words = []
        tags = []
        for line in file:
            if line.strip() == "":
                if words:
                    sentences.append(words)
                    labels.append(tags)
                    words = []
                    tags = []
            else:
                word, tag = line.strip().split()
                words.append(word)
                tags.append(tag)
        if words:
            sentences.append(words)
            labels.append(tags)
    return sentences, labels

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, ClassLabel, Sequence

def convert_to_dataset_dict(df):
    unique_tags = set()
    for tag in df['ner_tags']:
        unique_tags.update(tag)
    unique_tags = list(unique_tags)
    unique_tags = sorted(unique_tags, key=lambda x: (x[2:], x[:1]))
    
    datasets = DatasetDict()

    # Convertendo colunas para listas
    tokens = df['tokens'].tolist()
    ner_tags = df['ner_tags'].tolist()

    # Criando features de tokens e ner_tags
    features = {
        'tokens': Sequence(feature={"dtype": "string"}),
        'ner_tags': Sequence(feature=ClassLabel(num_classes=len(unique_tags), names=unique_tags))
    }

    # Criando dataset com um split único (neste caso, não estamos dividindo em treino/validação/teste)
    ner_tags = [list(map(lambda x: unique_tags.index(x), tags)) for tags in ner_tags]
    dataset = Dataset.from_dict({'tokens': tokens, 'ner_tags': ner_tags})
    dataset = dataset.map(lambda example: {'ner_tags': example['ner_tags']})

    datasets['train'] = dataset
    datasets.features = features

    return datasets

In [ ]:
import os

DIR_FILES = "/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/raw"

df_completo = pd.DataFrame()
for file in os.listdir(DIR_FILES):
    sentences, labels = read_conll(os.path.join(DIR_FILES, file))
    data = {'tokens': sentences, 'ner_tags': labels}
    df = pd.DataFrame(data)
    df_completo = pd.concat([df_completo, df], ignore_index=True)
# Pegar só a metade do dataframe completo
indice = len(df_completo) // 2

df_treino = df_completo.iloc[:indice]
df_teste = df_completo.iloc[indice:]

df_treino.dropna(inplace=True)
df_teste.dropna(inplace=True)

df_treino.reset_index(drop=True, inplace=True)
df_teste.reset_index(drop=True, inplace=True)

df_treino_dict = convert_to_dataset_dict(df_treino)
df_teste_dict = convert_to_dataset_dict(df_teste)

In [ ]:
df_treino.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_treino.csv", index=False)
df_teste.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_teste.csv", index=False)

In [ ]:
import json

df_treino_dict.save_to_disk("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_treino_dataset")
json.dump(df_treino_dict.features['ner_tags'].feature.names, open("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_treino_dataset/ner_pt_treino_dataset_features.json", "w"))

df_teste_dict.save_to_disk("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_teste_dataset")
json.dump(df_teste_dict.features['ner_tags'].feature.names, open("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt_teste_dataset/ner_pt_teste_dataset_features.json", "w"))